In [1]:
import pandas as pd
import numpy as np
from glob import glob
import os

In [2]:

file_lst = glob("raw/*.csv")


In [3]:
class RES:
    def __init__(self, features=[]):
        self.ft_n = len(features)
        self.features=features
        self.RES = np.empty((1, self.ft_n))
        self.RES_len=0
        self.RES_cap=1

    def add(self,x):
        if isinstance(x, pd.DataFrame):
            x = x.to_numpy()        
        size = x.shape[0]
        if (size + self.RES_len > self.RES_cap):
            while (size + self.RES_len > self.RES_cap):
                self.RES_cap *= 2
            
            newRes = np.empty((self.RES_cap, self.ft_n))
            newRes[0:self.RES_len,:] = self.RES[0:self.RES_len,:]
            self.RES = newRes
        self.RES[self.RES_len:self.RES_len+size, :] = x
        self.RES_len += size

    def getDf(self):
        return pd.DataFrame(self.RES[0:self.RES_len,:], columns=self.features)

    def __len__(self):
        return self.RES_len
        
    def __str__(self):
        return self.RES.__str__()


In [4]:

n_normal = 0
n_attack = 0
req_ft=['duration','src_bytes','dst_bytes','missed_bytes','src_pkts','dst_pkts','label']
print(len(req_ft))
r = RES(req_ft)
for f in file_lst:    
    df = pd.read_csv(f, low_memory=False)  
    df = df[df['type'].isin(['normal', 'dos', 'ddos'])]
    df = df.loc[:, df.columns.isin(req_ft)]
    size = df.shape[0]
    
    at_df = df[df['label'] == 1]
    nr_df = df[df['label'] == 0]
    n_nr = nr_df.shape[0]
    n_normal += n_nr
    n_attack += at_df.shape[0]    

    print(len(r))    
    
    rng = np.random.default_rng()
    index=np.sort(rng.choice([i for i in range(size)], size=int(size*0.1), replace=False, shuffle=False))
    r.add(df.to_numpy()[index])
    del df
    


7
0
300000
600000
900000
904322
1166763
1466763
1473087
1474184
1536787
1738547
2038547
2222068
2223783
2224629
2226878
2526878
2597374
2781236
3081236
3083112
3084590


In [5]:
print (f"NORMAL: {n_normal} ATTACK: {n_attack}")

NORMAL: 755124 ATTACK: 9540336


In [6]:
df = r.getDf()
if not os.path.exists('processed'):
        os.makedirs('processed')    
    
df.to_csv("processed/train_data.csv", index=False)

In [7]:
df.tail(15)

,duration,src_bytes,dst_bytes,missed_bytes,src_pkts,dst_pkts,label
3088616,0.000466,0.0,178.0,0.0,0.0,2.0,0.0
3088617,0.000501,0.0,212.0,0.0,0.0,2.0,0.0
3088618,0.000375,0.0,178.0,0.0,0.0,2.0,0.0
3088619,0.409033,44.0,0.0,0.0,2.0,0.0,0.0
3088620,0.000497,0.0,212.0,0.0,0.0,2.0,0.0
3088621,0.000447,31.0,106.0,0.0,1.0,1.0,0.0
3088622,0.000514,31.0,106.0,0.0,1.0,1.0,0.0
3088623,0.001996,0.0,178.0,0.0,0.0,2.0,0.0
3088624,0.000591,0.0,212.0,0.0,0.0,2.0,0.0
3088625,0.000618,0.0,212.0,0.0,0.0,2.0,0.0
